In [37]:
import pandas as pd
from tqdm import tqdm
import os

In [38]:
df_mapping_no2_to_traffico = pd.read_csv("../../data/processed/mapping/no2_to_traffic_sensor_mapping.csv")

In [39]:
import pandas as pd
from tqdm import tqdm

def process_traffic_data(df_traffic):
    """
    Función para procesar datos de tráfico a nivel horario a partir de datos cada 15 minutos.
    
    Se agrupan los datos por sensor y hora, calculando:
    - **Intensidad**: Promedio de la intensidad (vehículos/hora).
    - **Carga**: Promedio ponderado de la carga, usando la intensidad como peso.
    - **Ocupación**: Promedio ponderado de la ocupación, usando la intensidad como peso.
    - **Velocidad media (vmed)**: Promedio ponderado de la velocidad media, usando la intensidad como peso.

    Parámetros:
        df_traffic (pd.DataFrame): DataFrame con datos de tráfico a nivel de 15 minutos.

    Retorna:
        pd.DataFrame: DataFrame con los datos agregados a nivel horario.
    """

    # Convertir la columna 'fecha' a tipo datetime para manipulación de fechas
    df_traffic['fecha'] = pd.to_datetime(df_traffic['fecha'])

    # Redondear la fecha a la hora más cercana (para agrupar los datos por hora)
    df_traffic['hora'] = df_traffic['fecha'].dt.floor('H')

    # Rellenar valores NaN con -1 para evitar problemas en cálculos posteriores
    df_traffic = df_traffic.fillna(-1)

    # Función para calcular un promedio ponderado
    def weighted_average(group, value_column, weight_column):
        """
        Calcula la media ponderada de una columna usando otra como peso.

        Parámetros:
            group (pd.DataFrame): Grupo de datos a procesar.
            value_column (str): Nombre de la columna cuyo promedio ponderado se calcula.
            weight_column (str): Nombre de la columna que se usa como peso.

        Retorna:
            float: Media ponderada.
        """
        values = group[value_column]
        weights = group[weight_column]
        
        # Si la suma de los pesos es mayor a 0, se calcula la media ponderada, si no, se devuelve 0
        weighted_avg = (values * weights).sum() / weights.sum() if weights.sum() > 0 else 0
        return weighted_avg

    # Agrupar los datos por sensor y hora
    grouped = df_traffic.groupby(['id_trafico', 'hora'])

    # Lista para almacenar los resultados procesados
    result = []

    # Procesar cada grupo de sensor y hora
    for name, group in tqdm(grouped, desc="Processing groups"):
        """
        Para cada grupo (es decir, cada sensor en cada hora):
        - Se calcula la **intensidad** como promedio de la intensidad en los 4 intervalos de 15 minutos.
        - Se calcula la **carga** como un promedio ponderado de los valores de carga usando intensidad como peso.
        - Se calcula la **ocupación** como un promedio ponderado usando la intensidad como peso.
        - Se calcula la **velocidad media (vmed)** como un promedio ponderado usando la intensidad como peso.
        """

        # Intensidad: Como ya está en vehículos/hora, tomamos el promedio, no la suma
        intensidad_mean = group['intensidad'].mean()

        # Carga: Media ponderada con intensidad como peso
        carga_mean = weighted_average(group, 'carga', 'intensidad')

        # Ocupación: Media ponderada con intensidad como peso (no se puede sumar porque es un porcentaje)
        ocupacion_mean = weighted_average(group, 'ocupacion', 'intensidad')

        # Velocidad media: Media ponderada con intensidad como peso
        vmed_weighted = weighted_average(group, 'vmed', 'intensidad')

        # Agregar los resultados procesados a la lista
        result.append({
            'id_trafico': name[0],   # ID del sensor de tráfico
            'hora': name[1],         # Hora agregada
            'intensidad': intensidad_mean,  # Promedio de intensidad (veh/h)
            'carga': carga_mean,           # Promedio ponderado de carga
            'ocupacion': ocupacion_mean,   # Promedio ponderado de ocupación
            'vmed': vmed_weighted          # Promedio ponderado de velocidad media
        })

    # Convertir la lista de resultados en un DataFrame
    df_traffic = pd.DataFrame(result)

    return df_traffic  # Retornar el DataFrame procesado


In [40]:
def corregir_errores(df):
    # Asegurarse de que 'fecha' sea de tipo datetime
    
    # Iterar por las filas donde 'error' es 'S'
    for index, row in df[df['error'] == 'S'].iterrows():
        # Inicializar el contador de minutos
        minutos_atras = 15
        
        try:
            while True:
                # Buscar la fila anterior con el mismo 'id_trafico' y 'error' diferente de 'S'
                fila_anterior = df[
                    (df['id_trafico'] == row['id_trafico']) & 
                    (df['fecha'] == row['fecha'] - pd.Timedelta(minutes=minutos_atras))
                ]
                
                if not fila_anterior.empty and fila_anterior.iloc[0]['error'] != 'S':
                    # Copiar los valores de la fila anterior en la fila con error
                    for col in ['intensidad', 'ocupacion', 'carga', 'vmed', 'periodo_integracion']:
                        df.at[index, col] = fila_anterior.iloc[0][col]
                    
                    # Ajustar el error a 'N' después de la corrección
                    df.at[index, 'error'] = 'N'
                    break  # Salir del bucle si se ha encontrado una fila válida
                
                # Incrementar el contador de minutos para buscar más atrás
                minutos_atras += 15
                
                # Si se han buscado más de un número razonable de filas, salir para evitar bucles infinitos
                if minutos_atras > 180:  # Por ejemplo, buscar hasta 3 horas atrás
                    break
        except:
            df.at[index, col] = -1
        
    return df

In [41]:
df_mapping_no2_to_traffico['id_trafico'] = df_mapping_no2_to_traffico['id_trafico'].astype(int).astype(str)
filtered_traffic_sensors = list(df_mapping_no2_to_traffico.id_trafico.unique())

In [45]:
len(filtered_traffic_sensors)

69

In [46]:
# Define the root directory
root_dir = '../../data/raw/traffic'

# Iterate through all year folders
for year in range(2019, 2025):
    year_folder = os.path.join(root_dir, str(year))
    
    # Check if the year folder exists
    if os.path.isdir(year_folder):
        print(f"Processing folder: {year_folder}")
        
        # Iterate through files in the year folder
        for file_name in os.listdir(year_folder):
            if file_name.endswith('.csv'):  # Ensure it's a CSV file
                
                
                file_path = os.path.join(year_folder, file_name)
                
                try:
                    df = pd.read_csv(file_path, delimiter= ";")
                    
                    df = df.rename(columns = {'hora': 'fecha'})
                    df['fecha'] = pd.to_datetime(df['fecha'])
                        
                    # algunos se llaman id y otros identif
                    # Rename columns if they exist
                    if 'identif' in df.columns:
                        df = df.rename(columns={'identif': 'id_trafico'})

                    if 'id' in df.columns:
                        df = df.rename(columns={'id': 'id_trafico'})
                        
                    df['id_trafico'] = df['id_trafico'].apply(lambda x: str(int(x)) if str(x).isdigit() else str(x))
                    df['id_trafico'] = df['id_trafico'].astype(str)

                    print("len df before filtering ", len(df))
                    print("Unique traffic sensors before filtering:" , df.id_trafico.nunique())

                    df = df[df['id_trafico'].isin(filtered_traffic_sensors)]
                    
                    print("len df after filtering ", len(df))
                    print("Unique traffic sensors after filtering:" , df.id_trafico.nunique())
                    
                    if 'S' in df.error.unique():
                        print(f"There are errors in file: {file_name}")
                        print("There are a total of ", len(df[df['error'] == 'S']) , " errors")
                        
                        df = corregir_errores(df)
                        
                    df = process_traffic_data(df)
                        
                    # Create the new file name
                    new_file_name = f"{file_name[:-4]}_processed.parquet"
                    new_file_path = os.path.join(year_folder, new_file_name)
                    new_file_path = new_file_path.replace("raw", "processed")
                    
                    # Create the processed directory if it does not exist
                    processed_dir = os.path.dirname(new_file_path)
                    os.makedirs(processed_dir, exist_ok=True)

                    # Save the processed DataFrame
                    df.to_parquet(new_file_path, index=False)
                    
                    print(f"Processed and saved: {new_file_path}")
                except Exception as e:
                    print(f"Error processing file {file_name}: {e}")


Processing folder: ../../data/raw/traffic/2019
len df before filtering  11203627
Unique traffic sensors before filtering: 3937
len df after filtering  154550
Unique traffic sensors after filtering: 53


/var/folders/kc/t6c9n8gj7mbblj0w7sy3ykwm0000gn/T/ipykernel_17468/3807989780.py:25: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_traffic['hora'] = df_traffic['fecha'].dt.floor('H')
Processing groups: 100%|██████████| 38852/38852 [00:05<00:00, 7385.96it/s]


Processed and saved: ../../data/processed/traffic/2019/01-2019_processed.parquet
len df before filtering  11252731
Unique traffic sensors before filtering: 3998
len df after filtering  149099
Unique traffic sensors after filtering: 53


/var/folders/kc/t6c9n8gj7mbblj0w7sy3ykwm0000gn/T/ipykernel_17468/3807989780.py:25: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_traffic['hora'] = df_traffic['fecha'].dt.floor('H')
Processing groups: 100%|██████████| 37438/37438 [00:05<00:00, 7185.90it/s]


Processed and saved: ../../data/processed/traffic/2019/07-2019_processed.parquet
len df before filtering  10794005
Unique traffic sensors before filtering: 3982
len df after filtering  143596
Unique traffic sensors after filtering: 53


...

Processed and saved: ../../data/processed/traffic/2024/12-2024_processed.parquet
len df before filtering  13357035
Unique traffic sensors before filtering: 4706
len df after filtering  178987
Unique traffic sensors after filtering: 62


/var/folders/kc/t6c9n8gj7mbblj0w7sy3ykwm0000gn/T/ipykernel_17468/3807989780.py:25: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_traffic['hora'] = df_traffic['fecha'].dt.floor('H')
Processing groups: 100%|██████████| 45406/45406 [00:06<00:00, 7210.62it/s]


Processed and saved: ../../data/processed/traffic/2024/05-2024_processed.parquet
len df before filtering  13010780
Unique traffic sensors before filtering: 4695
len df after filtering  177288
Unique traffic sensors after filtering: 63


/var/folders/kc/t6c9n8gj7mbblj0w7sy3ykwm0000gn/T/ipykernel_17468/3807989780.py:25: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_traffic['hora'] = df_traffic['fecha'].dt.floor('H')
Processing groups: 100%|██████████| 44749/44749 [00:06<00:00, 7143.69it/s]


Processed and saved: ../../data/processed/traffic/2024/04-2024_processed.parquet


In [18]:
# df = pd.read_csv('../../data/raw/traffic/2020/01-2020.csv', delimiter= ";")
                    
# df = df.rename(columns = {'hora': 'fecha'})
# df['fecha'] = pd.to_datetime(df['fecha'])

#     # algunos se llaman id y otros identif
# # Rename columns if they exist
# if 'identif' in df.columns:
#     df = df.rename(columns={'identif': 'id_trafico'})

# if 'id' in df.columns:
#     df = df.rename(columns={'id': 'id_trafico'})
    
# df['id_trafico'] = df['id_trafico'].apply(lambda x: str(int(x)) if str(x).isdigit() else str(x))
# df['id_trafico'] = df['id_trafico'].astype(str)

# print(len(df))
# df = df[df['id_trafico'].isin(filtered_traffic_sensors)]
# print(len(df))


# if 'S' in df.error.unique():
#     print(f"There are errors in file: {file_name}")
#     print("There are a total of ", len(df[df['error'] == 'S']) , " errors")
    
#     df = corregir_errores(df)
    
# df = process_traffic_data(df)
    
# # Create the new file name
# new_file_name = f"{file_name[:-4]}_processed.parquet"
# new_file_path = os.path.join(year_folder, new_file_name)
# new_file_path = new_file_path.replace("raw", "processed")

# # Create the processed directory if it does not exist
# processed_dir = os.path.dirname(new_file_path)
# os.makedirs(processed_dir, exist_ok=True)

# # Save the processed DataFrame
# df.to_parquet(new_file_path, index=False)

# print(f"Processed and saved: {new_file_path}")

11577408
147118


/var/folders/kc/t6c9n8gj7mbblj0w7sy3ykwm0000gn/T/ipykernel_15168/1387776518.py:4: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_traffic['hora'] = df_traffic['fecha'].dt.floor('H')
Processing groups: 100%|██████████| 37202/37202 [00:04<00:00, 9020.19it/s]


NameError: name 'file_name' is not defined

In [36]:
from pyproj import Proj, Transformer

def latlon_to_utm(lat, lon):
    """
    Convierte coordenadas de latitud y longitud a coordenadas UTM (X, Y).
    
    Parámetros:
    - lat: Latitud en grados decimales.
    - lon: Longitud en grados decimales.
    
    Retorna:
    - utm_x: Coordenada X en UTM.
    - utm_y: Coordenada Y en UTM.
    - zona_utm: Número de la zona UTM.
    """
    # Determinar la zona UTM según la longitud
    zona_utm = int((lon + 180) / 6) + 1
    
    # Definir el sistema de referencia UTM correspondiente
    utm_proj = Proj(proj="utm", zone=zona_utm, ellps="WGS84", datum="WGS84", south=False)
    
    # Transformar coordenadas geográficas (lat, lon) a UTM (X, Y)
    utm_x, utm_y = utm_proj(lon, lat)
    
    return round(utm_x,3), round(utm_y,3), zona_utm

# Ejemplo de uso
	

lat, lon = -40.4305187, 3.687254201	
utm_x, utm_y, zona = latlon_to_utm(lat, lon)

print(f"UTM X: {round(utm_x,4)}, UTM Y: {round(utm_y,4)}, Zona: {zona}")


UTM X: 558293.961, UTM Y: -4475769.104, Zona: 31


In [34]:
round(-40.4305187, 4), round(3.68725420,4)

(-40.4305, 3.6873)

In [33]:
round(lat,4), round(lon,4)

(40.4305, -3.6873)

In [31]:
from pyproj import Proj, Transformer

def utm_to_latlon(utm_x, utm_y, zona_utm):
    """
    Convierte coordenadas UTM (X, Y) a latitud y longitud en España (Hemisferio Norte).
    
    Parámetros:
    - utm_x: Coordenada X en UTM.
    - utm_y: Coordenada Y en UTM.
    - zona_utm: Número de la zona UTM (España está en 29, 30 o 31).
    
    Retorna:
    - lat: Latitud en grados decimales.
    - lon: Longitud en grados decimales.
    """
    # Definir el sistema de referencia UTM correspondiente (siempre en el hemisferio norte)
    utm_proj = Proj(proj="utm", zone=zona_utm, ellps="WGS84", datum="WGS84", south=False)
    
    # Invertir la transformación de UTM a lat/lon
    lon, lat = utm_proj(utm_x, utm_y, inverse=True)
    
    return lat, lon

# Ejemplo de uso

utm_x, utm_y, zona = 441705.882339595, 4475769.68733175, 30
lat, lon = utm_to_latlon(utm_x, utm_y, zona)

print(f"Latitud: {lat}, Longitud: {lon}")


Latitud: 40.43052393961246, Longitud: -3.687256103050413


(40.4305, -3.6873)